In [ ]:
# ! /usr/bin/env python
# -*- coding=utf-8 -*-

import urllib
import urllib2
from urllib3.contrib import pyopenssl
import requests
import re,random
from multiprocessing.dummy import Pool as ThreadPool 
import re
#import urllib.request
#import urllib.error
import gzip
from bs4 import BeautifulSoup, element
#import mysql.connector
from collections import namedtuple
import os
import platform
import logging
import subprocess
import time
from datetime import datetime

time_out = 10 # 全局变量 10 秒超时时间
count = 0
proxies = [None]

Proxy = namedtuple("Proxy", ["ip", "port", "addtime", "wspeed", "lspeed", "country", "anonymity", "linuxping", "winping"])

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
             'Accept':'text/html;q=0.9,*/*;q=0.8',
             'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             #'Accept-Encoding':'gzip',
             'Connection':'close',
             'Referer':None #注意如果依然不能抓取的话，这里可以设置抓取网站的host
             }



def get_proxy():
    # 使用全局变量,修改之
    global proxies
    '''
    try:
        url = 'http://www.xicidaili.com/'
        #url = 'http://www.gatherproxy.com/proxylist/anonymity/?t=Elite'
        #url = 'https://free-proxy-list.net/anonymous-proxy.html'
        req_header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
             'Accept':'text/html;q=0.9,*/*;q=0.8',
             'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             #'Accept-Encoding':'gzip',
             'Connection':'close',
             'Referer':None #注意如果依然不能抓取的话，这里可以设置抓取网站的host
             }
        req_timeout = 10
        req = urllib2.Request(url,None,req_header)
        resp = urllib2.urlopen(req,None,req_timeout)
        html = resp.read()
     
        #req.add_header("User-Agent","Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 
        #  (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36")

        response = urllib2.urlopen(req)
        the_page = response.read()
        #req = urllib2.urlopen('http://www.xicidaili.com/',None,headers)
        #req = urllib2.Request("http://www.xicidaili.com/", headers=headers)
        #req  = urllib2.urlopen('http://www.xicidaili.com/')
        #print("===== start to get proxy list",req.read())
        
    except urllib2.URLError,e:
        print('无法获取代理信息!')
        print e.reason
        print e.reason[0]
        print e.reason[1]
        return
    
    
    response =urllib2.urlopen(req)
    print ("===== content-type : ",response.headers['content-type'] )
    print ("===== headers.getparam('charset') : ",response.headers.getparam('charset') )
    
    result = response.read().decode('utf-8')
    #html = response.read().decode('utf-8')
   
    #print("===== get proxy page ,type : ",result,type(result) 
    '''
    url ='https://free-proxy-list.net/anonymous-proxy.html'
    
    req = requests.get(url,headers=headers)

    #req=requests.post(url,headers=headers)

    #req.prepare().body
    #req.prepare().headers
    #url_result = requests.get(url)
    #print("===== req:,type ",req.text,type(req))
    #soup = BeautifulSoup(requests.get(url).content)
    soup = BeautifulSoup(req.text, "html.parser")
    proxylist = []
    print("begin getProxy")
    try:
        if url == "https://free-proxy-list.net/anonymous-proxy.html":
            tbody = soup.tbody
            #print("===== tbody:,type ",tbody,type(tbody))
            for child in tbody.children:
                if isinstance(child, element.Tag) and len(child) > 1:
                    # ip
                    tp = child.contents[0]
                    ip = str(tp.contents[0].string)
                    # port
                    try:
                        tp = child.contents[1]
                        port = str(tp.contents[0].string)
                    except IndexError as e:
                        logging.exception(e)
                        port = str(tp.string).strip()
                        print(port)
                    # time
                    addtime = datetime.utcnow()
                    # speed
                    wspeed = 9999
                    lspeed = 9999
                    # country
                    tp = child.contents[3]
                    country = str(tp.contents[0].strip())
                    # anonymity
                    tp = child.contents[4]
                    anonymity = str(tp.contents[0].string).lower()
                    winping = 9999
                    linuxping = 9999
                    proxy_item = Proxy(ip, port, addtime, wspeed, lspeed, country, anonymity, linuxping, winping)
                    proxylist.append(proxy_item)
                    #print("===== proxylist:,type ",proxylist,type(proxylist))
                    logging.info(proxy_item)
        else:
            pass
        print("finish getProxy")
    except Exception as e:
        logging.exception(e)
    
    #p = re.compile(r'''<tr\sclass[^>]*>\s+<td\s.*</td>\s+<td>(.*)?</td>\s+<td>(.*)?</td>\s+<td>.*</td>\s+<td.*</td>\s+<td>(.*)?</td>\s+<td>.*</td>\s+<td>.*</td>\s+</tr>''',re.VERBOSE)
    #proxy_list = p.findall(result)
    
    #proxy_list = re.findall(r'<tr class="odd"><</tr>',result, re.M|re.I|re.S)
    
    
    #print("===== get proxy page proxy_list:,type ",proxy_list,type(proxy_list))
    proxies.append('221.211.193.51:80')
    proxies.append('120.7.84.59:8118')
    proxies.append('203.91.121.76:3128')
    proxies.append('180.168.179.193:8080')
    for each_proxy in proxylist[1:]:
        proxies.append(each_proxy[0]+':'+each_proxy[1])
    print("===== proxies:,type ",proxies,type(proxies))  
    
def change_proxy():
    # 随机从序列中取出一个元素
    proxy = random.choice(proxies)
    # 判断元素是否合理
    if proxy == None:
        proxy_support =urllib2.ProxyHandler({})
    else:
        proxy_support = urllib2.ProxyHandler({'http':proxy})
    opener = urllib2.build_opener(proxy_support)
    opener.addheaders = [('User-Agent',headers['User-Agent'])]
    urllib2.install_opener(opener)
    print('智能切换代理：%s' % ('本机' if proxy==None else proxy))
def get_req(url):
    # 先伪造一下头部吧,使用字典
    blog_eader = {
                'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.152 Safari/537.36',
                'Host':'blog.csdn.net',
                'Referer':'http://blog.csdn.net/',
                'GET':url
                } 
    #req = urllib2.urlopen(url,headers = blog_eader)
    req = urllib2.Request(url, headers=headers)
    return req
# 访问 博客
def look_blog(url):
    # 切换一下IP
    change_proxy()
    req = get_req(url)
    try:
        look_response = urllib2.urlopen(req,timeout = time_out)
    except:
        return
    else:
        print('Access Successfully !')
        print ('Look result :',look_response.read())
        #return look_response 
# 迭代访问
def click_blog(url):
    for i in range(0,count):
        if(i == count):
            break
        print('当前访问 Blog %s 第 %d 次' % (url,i))
        look_blog(url)

if __name__ == '__main__':
    global count
    # 基本参数初始化
    # 获取代理
    print("===initialization")
    get_proxy()
    print('有效代理个数为 : %d' % len(proxies))
    #blogurl = input('输入blog链接:')
    # 这个地方原本是我的默认输入偷懒用的
    #blogurl = 'https://youtu.be/RysyB_Zcjlo'
    blogurl = 'http://icanhazip.com/'
    if len(blogurl) == 0:
        #blogurl = 'http://blog.csdn.net/bkxiaoc/'
        blogurl = 'https://youtu.be/RysyB_Zcjlo'
        
    print('URL is :' ,blogurl)
    try:
        count = int(input('输入次数:'))
    except ValueError:
        print('参数错误')
        quit() 
    if count == 0 or count > 999:
        print('次数过大或过小')
        quit()
    print('次数确认为 %d' % count)
    # 获取 博文 列表,由于测试时我的博文只有一页所以 只能获得一页的列表
    #blog_list = get_blog_list("https://youtu.be/RysyB_Zcjlo")
    #print ("==== blog_list , type ",blog_list, type(blog_list))
    #if len(blog_list) == 0:
    #    print('未找到Blog列表')
    #    quit()
    print('启动!!!!!!!!!!!!!!!!!!!!')
    # 迭代一下 使用多线程
    
    index = 0
    url_list = []
    #maxConnection = 10 
    for i in range(10):
        # 补全头部
        each_link = blogurl + '?' + str(i)
        print ("===== each_link, type",each_link,type(each_link))
        print ("===== index, type",index,type(index))
        
        url_list.append(each_link)
        index += 1
    # 有多少个帖子就开多少个线程的一半 let's go
    pool = ThreadPool(15)
    batch_result = pool.map(click_blog, url_list)
    print ("===== batch_result, type",batch_result,type(batch_result))
    pool.close()
    pool.join()
    print('完成任务!!!!!!!!!!!!!!!!!!!!')
    
    
    
    

===initialization
begin getProxy
finish getProxy
('===== proxies:,type ', [None, '221.211.193.51:80', '120.7.84.59:8118', '203.91.121.76:3128', '180.168.179.193:8080', '178.170.185.171:53281', '116.72.50.122:3128', '109.121.161.192:53281', '103.67.198.228:53281', '14.203.99.67:8080', '106.186.121.159:9999', '216.56.48.118:9000', '81.10.172.142:3128', '178.49.136.84:8080', '171.101.237.248:3128', '94.177.175.232:3128', '203.104.192.230:3128', '45.79.0.108:1080', '216.105.171.46:3128', '103.87.16.2:80', '142.75.2.14:3128', '05.135.195.166:3128', '175.209.34.197:8080', '186.227.8.21:3128', '213.136.89.121:80', '213.136.77.246:80', '47.89.41.164:80', '118.97.129.219:8080', '104.198.227.184:80', '195.154.42.249:3128', '45.32.181.140:3128', '50.232.30.218:3128', '125.24.120.168:8080', '203.58.117.34:80', '103.199.119.212:8080', '210.16.121.16:3128', '168.195.210.183:65205', '179.191.12.24:80', '189.206.107.6:8080', '80.83.20.14:80', '173.161.162.68:8118', '213.183.146.70:9999', '187.108.38.2

In [1]:
print ("123")

123
